In [1]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
%matplotlib inline

In [3]:
tf.random.set_seed(272)
img_size = 400
vgg = tf.keras.applications.VGG19(include_top=False,
                                  input_shape=(img_size, img_size, 3),
                                  weights='../input/artneural/model/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

vgg.trainable = False

In [4]:
vgg.summary()

In [5]:
content_path = "../input/artneural/content/louvre.jpg"
content_image = Image.open(content_path)
content_image

In [6]:
def compute_content_cost(content_output, generated_output):

    a_C = content_output[-1]
    a_G = generated_output[-1]

    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_C_unrolled = tf.reshape(a_C, [m, -1, n_C])
    a_G_unrolled = tf.reshape(a_G, [m, -1, n_C])
    
    J_content = (.25 / float(int(n_H * n_W * n_C))) * tf.reduce_sum(tf.square(tf.subtract(a_G_unrolled , a_C_unrolled)))
    
    return J_content

In [7]:
style_path = "../input/artneural/style/starry-night.jpg"
style = Image.open(style_path)
style

In [8]:
def gram_matrix(A):
    GA = tf.matmul(A, tf.transpose(A))
    return GA

In [9]:
def compute_layer_style_cost(a_S, a_G):

    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_S = tf.transpose(tf.reshape(a_S, [n_H * n_W, n_C]))
    a_G = tf.transpose(tf.reshape(a_G, [n_H * n_W, n_C]))

    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)

    J_style_layer = ((.5 / (n_H * n_W * n_C)) ** 2) * tf.reduce_sum(tf.square(tf.subtract(GS , GG)))

    return J_style_layer

In [10]:
for layer in vgg.layers:
    print(layer.name)

In [12]:
STYLE_LAYERS = [
    ('block1_conv1', 0.35),
    ('block2_conv1', 0.5),
    ('block3_conv1', 0.05),
    ('block4_conv1', 0.05),
    ('block5_conv1', 0.05)
]

In [13]:
def compute_style_cost(style_image_output, generated_image_output, STYLE_LAYERS=STYLE_LAYERS):

    J_style = 0
    a_S = style_image_output[:-1]

    a_G = generated_image_output[:-1]
    for i, weight in zip(range(len(a_S)), STYLE_LAYERS):  
        J_style_layer = compute_layer_style_cost(a_S[i], a_G[i])
        J_style += weight[1] * J_style_layer

    return J_style

In [14]:
@tf.function()
def total_cost(J_content, J_style, alpha = 10, beta = 40):
    J = alpha * J_content + beta * J_style
    return J

In [15]:
content_image = np.array(Image.open(content_path).resize((img_size, img_size)))
content_image = tf.constant(np.expand_dims(content_image,axis=0))

print(content_image.shape)
plt.imshow(content_image[0])
plt.show()

In [16]:
style_image =  np.array(Image.open(style_path).resize((img_size, img_size)))
style_image = tf.constant(np.expand_dims(style_image,axis=0))

print(style_image.shape)
plt.imshow(style_image[0])
plt.show()

In [40]:
generated_image = tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
noise = tf.random.uniform(tf.shape(generated_image), -0.3, 0.3)
generated_image = tf.add(generated_image, noise)
generated_image = tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=1.0)

print(generated_image.shape)
plt.imshow(generated_image.numpy()[0])
plt.show()

In [18]:
content_layer = [('block5_conv4', 1)]
outputs = [vgg.get_layer(layer[0]).output for layer in STYLE_LAYERS+content_layer]
vgg_model = tf.keras.Model([vgg.input], outputs)

In [19]:
vgg_model.output

In [41]:
preprocessed_content =  tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
a_C = vgg_model(preprocessed_content)

preprocessed_style =  tf.Variable(tf.image.convert_image_dtype(style_image, tf.float32))
a_S = vgg_model(preprocessed_style)

In [21]:
def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        tensor = tensor[0]
    return Image.fromarray(tensor)

In [42]:
generated_image = tf.Variable(generated_image)

In [44]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
epochs = 20001

@tf.function()
def train(generated_image, C, S):
    with tf.GradientTape() as tape:
        a_G = vgg_model(generated_image)
        J_style = compute_style_cost(S,a_G,STYLE_LAYERS)
        J_content = compute_content_cost(C,a_G)
        J = total_cost(J_content,J_style, alpha=10, beta=40)
    grad = tape.gradient(J, generated_image)
    optimizer.apply_gradients([(grad, generated_image)])
    generated_image.assign(clip_0_1(generated_image))
    return J

In [45]:
for i in range(epochs):
    train(generated_image, a_C, a_S)
    if i % 2000 == 0:
        print(f"Epoch {i} ")
        image = tensor_to_image(generated_image)
        plt.imshow(image)
        plt.show() 

In [46]:
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(1, 3, 1)
plt.imshow(content_image[0])
ax.title.set_text('Content image')
ax = fig.add_subplot(1, 3, 2)
plt.imshow(style_image[0])
ax.title.set_text('Style image')
ax = fig.add_subplot(1, 3, 3)
plt.imshow(generated_image[0])
ax.title.set_text('Generated image')
plt.show()

In [26]:
os.mkdir('./out')

In [27]:
image = tensor_to_image(generated_image)
image.save(f'./out/art_{style_path[25:]}')

In [50]:
fig.savefig('./out/fig-starry-night.png')

In [51]:
def generate_art(c_path, s_path):
    c_image = np.array(Image.open(c_path).resize((img_size, img_size)))
    c_image = tf.constant(np.expand_dims(c_image,axis=0))
    preprocessed_content_img =  tf.Variable(tf.image.convert_image_dtype(c_image, tf.float32))
    a_C_target = vgg_model(preprocessed_content_img)

    s_image = np.array(Image.open(s_path).resize((img_size, img_size)))
    s_image = tf.constant(np.expand_dims(s_image,axis=0))
    preprocessed_style_img =  tf.Variable(tf.image.convert_image_dtype(s_image, tf.float32))
    a_S_target = vgg_model(preprocessed_style_img)
    
    g_image = tf.Variable(tf.image.convert_image_dtype(c_image, tf.float32))
    noise = tf.random.uniform(tf.shape(g_image), -0.3, 0.3)
    g_image = tf.add(g_image, noise)
    g_image = tf.clip_by_value(g_image, clip_value_min=0.0, clip_value_max=1.0)
    g_image = tf.Variable(g_image)
    
    @tf.function()
    def train2(g, C, S):
        with tf.GradientTape() as tape:
            a_G = vgg_model(g)
            J_style = compute_style_cost(S,a_G,STYLE_LAYERS)
            J_content = compute_content_cost(C,a_G)
            J = total_cost(J_content,J_style, alpha=10, beta=40)
        grad = tape.gradient(J, g_image)
        optimizer.apply_gradients([(grad, g_image)])
        g_image.assign(clip_0_1(g_image))
        return J
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    epochs = 22001
    for i in range(epochs):
        train2(g_image, a_C_target, a_S_target)
        
    fig = plt.figure(figsize=(16, 4))
    ax = fig.add_subplot(1, 3, 1)
    plt.imshow(c_image[0])
    ax.title.set_text('Content image')
    ax = fig.add_subplot(1, 3, 2)
    plt.imshow(s_image[0])
    ax.title.set_text('Style image')
    ax = fig.add_subplot(1, 3, 3)
    plt.imshow(g_image[0])
    ax.title.set_text('Generated image')
    plt.show()
    
    final_image = tensor_to_image(g_image)
    final_image.save(f'./out/art_{s_path[25:]}')
    
    return final_image, fig

In [48]:
c_path = '../input/artneural/content/cat.jpg'
s_path = '../input/artneural/style/guernica.jpg'
generate_art(c_path, s_path)

In [52]:
c_path = '../input/artneural/content/cat.jpg'
s_path = '../input/artneural/style/scream.jpg'
_, final_fig = generate_art(c_path, s_path)

In [53]:
_

In [55]:
final_fig.savefig('./out/fig-scream.png')

In [56]:
c_path = '../input/artneural/style/drop-of-water.jpg'
s_path = '../input/artneural/style/starry-night.jpg'
img2, fig2 = generate_art(c_path, s_path)

In [57]:
img2

In [58]:
fig2.savefig('./out/fig-water-starry.png')
img2.save('./out/staray.jpg')